In [2]:
import pandas as pd
df = pd.read_csv('./../../../text_emotion.csv')
df.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [3]:
#converting 

from io import StringIO
col = ['tweet_id','sentiment', 'content']
df = df[col]
df = df[pd.notnull(df['content'])]
# df = df.head(10000)
df.columns = ['tweet_id','sentiment', 'content']
df['category_id'] = df['sentiment'].factorize()[0]
category_id_df = df[['sentiment', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'sentiment']].values)
df.head()

,tweet_id,sentiment,content,category_id
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...,0
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...,1
2,1956967696,sadness,Funeral ceremony...gloomy friday...,1
3,1956967789,enthusiasm,wants to hang out with friends SOON!,2
4,1956968416,neutral,@dannycastillo We want to trade with someone w...,3


In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, SpatialDropout1D
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [6]:
max_fatures = 8000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['content'].values)
X = tokenizer.texts_to_sequences(df['content'].values)
X = pad_sequences(X)

NameError: name 'X_train' is not defined

In [7]:
Y = pd.get_dummies(df['category_id']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 0)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(36000, 35) (36000, 13)
(4000, 35) (4000, 13)


In [8]:
embed_dim = 50
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X_train.shape[1]))
# model.add(SpatialDropout1D(0.4))
model.add(SimpleRNN(units=8, input_shape=(X_train.shape[0], 
    X_train.shape[1])))
model.add(Dense(13,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])


Epoch 1/20
 - 19s - loss: 2.0932 - acc: 0.2756
Epoch 2/20


KeyboardInterrupt: 

In [30]:
validation_size = 500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
print(X_train.shape)
print(Y_test.shape)

print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

(8000, 34)
(1500, 13)
score: 2.37
acc: 0.24


In [ ]:
history=model.fit(X_train, Y_train, epochs=20, verbose = 2, batch_size=16)

In [ ]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = 16)